<img src="https://docs.xarray.dev/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# A Tour of Xarray Customizations

Xarray is easily extensible.
This means it is easy to add on to to build custom packages that tackle particular computational problems or supply domain specific functionality.

These packages can plug in to xarray in various different ways. They may build directly on top of xarray, or they may take advantage of some of xarray's dedicated interfacing features:
- Accessors (extensions)
- Backend (filetype) entrypoint
- Metadata attributes
- Duck-array wrapping interface
- Flexible indexes (coming soon!)

Here we introduce several popular or interesting extensions that are installable as their own packages (via conda and pip). These packages integrate with xarray using one or more of the features mentioned above.

- [hvplot](https://hvplot.holoviz.org/), a powerful interactive plotting library
- [rioxarray](https://corteva.github.io/rioxarray/stable/index.html), for working with geospatial raster data using rasterio
- [cf-xarray](https://cf-xarray.readthedocs.io/en/latest/), for interpreting CF-compliant data
- [pint-xarray](https://pint-xarray.readthedocs.io/en/latest/), for unit-aware computations using pint.

Specific examples for implementing your own xarray customizations using these interfacing features are available in the ADVANCED section of this book.

In [ ]:
import xarray as xr
import numpy as np

## Quick note on accessors

Before we look at the packages we need to briefly introduce a feature they commonly use: ["xarray accessors"](https://docs.xarray.dev/en/stable/internals/extending-xarray.html).

The accessor-style syntax is used heavily by the other libraries we are about to cover.

For users, accessors just allow us to have a familiar dot (method-like) syntax on xarray objects, for example `da.hvplot()`, `da.pint.quantify()`, or `da.cf.describe()`.


## hvplot via accessors

The [HoloViews library](https://holoviews.org/) makes great use of accessors to allow seamless plotting of xarray data using a completely different plotting backend (by default, xarray uses [matplotlib](https://matplotlib.org/)).

We first need to import the code that registers the hvplot accessor

In [ ]:
import hvplot.xarray

And now we can call the `.hvplot` method to plot using holoviews in the same way that we would have used `.plot` to plot using matplotlib.

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")

In [ ]:
ds['air'].isel(time=1).hvplot(cmap="fire")

For some more examples of how powerful HoloViews is [see here](https://tutorial.xarray.dev/intermediate/hvplot.html).

## Rioxarray via the backend entrypoint

[Rioxarray](https://corteva.github.io/rioxarray/stable/index.html) is a Python library that enhances Xarray's ability to work with geospatial data and coordinate reference systems. Geographic information systems use GeoTIFF and [many other formats](https://gdal.org/drivers/raster/index.html) to organize and store gridded, or *raster*, datasets. 

The Geospatial Data Abstraction Library ([GDAL](https://gdal.org)) provides foundational drivers and geospatial algorithms, and the [rasterio](https://rasterio.readthedocs.io/en/latest) library provides a Pythonic interface to GDAL. `Rioxarray` brings key features of rasterio to Xarray:

1. A backend *engine* to read any format recognized by *GDAL*
1. A `.rio` *accessor* for *rasterio's* geospatial algorithms such as reprojection

Below a couple brief examples to illustrate these features:

In [ ]:
import rioxarray  # ensure you have rioxarray installed in your environment

You can explicitly use rioxarray's 'rasterio' engine to load myriad geospatial raster formats. Below is a [Cloud-Optimized Geotiff](https://www.cogeo.org) from an AWS [public dataset](https://registry.opendata.aws/sentinel-1-rtc-indigo/) of synthetic aperture radar data over Washington, State, USA.  `overview_level=4` is an argument specific to the `rasterio` engine that allows opening a pre-computed lower resolution "overview" of the data.

In [ ]:
url = 'https://sentinel-s1-rtc-indigo.s3.us-west-2.amazonaws.com/tiles/RTC/1/IW/10/U/CU/2017/S1A_20170101_10UCU_ASC/Gamma0_VV.tif'
da = xr.open_dataarray(url, engine='rasterio', open_kwargs={"overview_level": 2})
da

The `spatial_ref` coordinate is added by rioxarray to store standardized geospatial Coordinate Reference System (CRS) information. We can access that information and additional methods via the `.rio` accessor:

In [ ]:
da.rio.crs

EPSG refers to 'European Petroleum Survey Group', a database of the many CRS definitions for our Planet used over the years! EPSG=32610 is the ["UTM 10N" CRS](https://epsg.io/32610), with coordinate units in meters. Let's say you want longitude,latitude coordinate points in degrees instead. You'd have to *reproject* this data:

In [ ]:
da_lonlat = da.rio.reproject('epsg:4326')
da_lonlat

Note that that the size of the data has changed as well as the coordinate values. This is typical of reprojection, as your data must be resampled and often interpolated to match the new CRS grid! A quick plot will compare the results of our reprojected data:

In [ ]:
import panel as pn

img1 = da.sel(band=1).hvplot.image(
    x='x', y='y', rasterize=True, cmap='gray', clim=(0, 0.5), title='UTM'
)
img2 = da_lonlat.sel(band=1).hvplot.image(
    rasterize=True, cmap='gray', clim=(0, 0.5), title='LON/LAT'
)

pn.Column(img1, img2)

## cf-xarray via metadata attributes

Xarray objects can store [arbitrary metadata](https://docs.xarray.dev/en/stable/getting-started-guide/faq.html#what-is-your-approach-to-metadata) in the form of a `dict` attached to each `DataArray` and `Dataset` object, accessible via the `.attrs` property.

In [ ]:
xr.DataArray(name="Hitchhiker", data=0, attrs={"life": 42, "name": "Arthur Dent"})

Normally xarray operations ignore this metadata, simply carting it around until you explicitly choose to use it. However sometimes we might want to write custom code which makes use of the metadata.

[cf_xarray](https://cf-xarray.readthedocs.io/) is a project that tries to
let you make use of other [Climate and Forecast metadata convention attributes](http://cfconventions.org/) (or "CF attributes") that xarray ignores. It attaches itself
to all xarray objects under the `.cf` namespace.

Where xarray allows you to specify dimension names for analysis, `cf_xarray`
lets you specify logical names like `"latitude"` or `"longitude"` instead as
long as the appropriate CF attributes are set.

For example, the `"longitude"` dimension in different files might be labelled as: (lon, LON, long, x…), but cf_xarray let's you always refer to the logical name `"longitude"` in your code:

In [ ]:
import cf_xarray

In [ ]:
# describe cf attributes in dataset
ds.air.cf.describe()

The following `mean` operation will work with any dataset that has appropriate
attributes set that allow detection of the "latitude" variable (e.g.
`units: "degress_north"` or `standard_name: "latitude"`)


In [ ]:
# demonstrate equivalent of .mean("lat")
ds.air.cf.mean("latitude")

In [ ]:
# demonstrate indexing
ds.air.cf.sel(longitude=242.5, method="nearest")

## Pint via duck array wrapping

### Why use pint

[Pint](https://pint.readthedocs.io/en/stable/) defines physical units, allowing you to work with numpy-like arrays which track the units of your values through computations.

Pint defines a numpy-like array class called `pint.Quantity`, which is made up of a numpy array and a `pint.Unit` instance.

In [ ]:
from pint import Unit

# you can create a pint.Quantity by multiplying a value by a pint.Unit
d = np.array(10) * Unit("metres")
d

These units are automatically propagated through operations

In [ ]:
t = 1 * Unit("seconds")
v = d / t
v

Or if the operation involves inconsistent units, a `pint.DimensionalityError` is raised.

In [ ]:
d + t

### pint inside xarray objects

We have already seen that xarray can wrap numpy arrays or dask arrays, but in fact xarray can wrap any type of array that behaves similarly to a numpy array.
Using this feature we can store a `pint.Quantity` array inside an xarray DataArray

In [ ]:
da = xr.DataArray(d)
da

We can see that the data type stored within the DataArray is a `Quantity` object, rather than just a `np.ndarray` object, and the units of the data are displayed in the repr.

The reason this works is that a `pint.Quantity` array is what we call a "duck array", in that it behaves so similarly to a `numpy.ndarray` that xarray can treat them the same way. (See [python duck typing](https://towardsdatascience.com/duck-typing-python-7aeac97e11f8))

### pint-xarray

The convenience package [pint-xarray](https://pint-xarray.readthedocs.io/en/latest/index.html) makes it easier to get the benefits of pint whilst working with xarray objects.

It provides utility accessor methods for promoting xarray data to pint quantities (which we call "quantifying") in various ways and for manipulating the resulting objects.

In [ ]:
# to be able to read unit attributes following the CF conventions
import cf_xarray.units
import pint_xarray

xr.set_options(display_expand_data=False)

pint-xarray provides the `.pint` accessor, which firstly allows us to easily extract the units of our data

In [ ]:
da.pint.units

The `.pint.quantify()` accessor gives us various ways to convert normal xarray data to be unit-aware.

Note: It is preferred to use `.pint.quantify()` to convert xarray data to use pint rather than explicitly creating a `pint.Quantity` array and placing it inside the xarray object, because pint-xarray will handle various subtleties involving dask etc.

We can explicitly specify the units we want

In [ ]:
da = xr.DataArray([4.5, 6.7, 3.8], dims="time")
da.pint.quantify("V")

Or if the data has a "units" entry in its `.attrs` metadata dictionary, we can automatically use that to convert each variable.

For example, the xarray tutorial dataset we opened earlier has units in its attributes

In [ ]:
ds.air.attrs['units']

which we can automatically read with `.pint.quantify()`:

In [ ]:
quantified_air = ds.pint.quantify()
quantified_air

Here we have actually gone even further, and used cf-xarray to automatically interpret cf-compliant units in the `.attrs` to valid pint units.
This automatically happened just as a result of importing cf-xarray above.

When we plot quantified data with xarray the correct units will automatically appear on the plot

In [ ]:
quantified_air["air"].isel(time=500).plot();

If we want to cast the pint arrays back to numpy arrays, we can use `.pint.dequantify()`, which will also write the current units back out to the `.attrs["units"]` field

In [ ]:
quantified_air.pint.dequantify()

**Exercise:** Write a function which will raise an error if supplied with data in the wrong units.

In [ ]:
from pint import DimensionalityError


def special_science_function(distance):
    if distance.pint.units != "miles":
        raise DimensionalityError(
            "this function will only give the correct answer if the input is in units of miles"
        )

**Exercise:** Try this on some of your data!

After you have imported pint-xarray (and maybe cf-xarray) as above, start with something like

`ds = xr.open_dataset(my_data).pint.quantify()`

Take a look at the [pint-xarray documentation](https://pint-xarray.readthedocs.io/en/latest/) or the [pint documentation](https://pint.readthedocs.io/en/stable/) if you get stuck.

## The wider world...

There are many other libraries in the wider xarray ecosystem. For a list of a few packages we particularly like for geoscience work [see here](https://tutorial.xarray.dev/overview/xarray-in-45-min.html#other-cool-packages), and for a [more exhaustive list see here](https://docs.xarray.dev/en/stable/ecosystem.html).